In [188]:
#IMPORTS

import enchant  #pip install pyenchant
import sys,os
import getpass
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools
import ast
import nltk 
#pip install nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [146]:
def extractWordAfterSymbol(text,symbol):
    
    """ 
    Generalizando, como hizo mauro
    
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: Una lista de todos los strings que estaban precedidos por un simbolo(sea @, #, etc)
    EJEMPLO: 'Hola que tal soy @axel, y vivo en @capitalFederal' -----> [axel,capitalFederal]'
    """
    
    words = text.split(' ')
    
    users = []
    userToSave = ""
    for word in words:
        if len(word) != 0 and symbol in word:
            splittedWord = word.split(symbol)
            if len(splittedWord) > 1:
                userToSave = splittedWord[1]
            else:
                userToSave = splittedWord[0]
        if userToSave != '':
            users.append(userToSave)
            userToSave = ""
    return users

In [147]:
def removeDots(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: La parte de la palabra que no tiene ningun punto
    EJEMPLO: 'esperando...' -----> 'esperando'
    """
    
    splited = word.split('.')
    
    for chunk in splited:
        if chunk != '.':
            return chunk
    return word

In [148]:
def hasNumeric(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: True o False dependiendo de si contiene algun numero
    EJEMPLO: 'hol4' -----> True
    """    
    
    for c in word:
        if c.isnumeric():
            return True
    return False

In [149]:
def getEnlgishWord(word):
       
    """
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: La una tupla con la palabra mas larga en ingles que encuentre primero dentro del texto recibido
              y un True o False dependiendo de si habia o no una palabra en ingles en el texto. 
              Si no contiene ninguna palabra en ingles devuelve el texto original.
    TENER EN CUENTA: No considera como una palabra valida a las que tengan longitud 1
    EJEMPLO: 'aaaaaaaaaaENGLISHaaaaaaaaaa' -----> 'ENGLISH'
    EJEMPLO: 'aaaaaaaaaaaaaaaaaaaa' -----> 'aaaaaaaaaaaaaaaaaaaa'
    """  
    
    englishDictionary = enchant.Dict("en_US")
    
    MIN_LEN_WORDS = 2
    
    failure = False
    
    if word == None:
        failure = True
    
    elif len(word) < MIN_LEN_WORDS:
        failure = True
        
    else:
        for i in range(len(word), MIN_LEN_WORDS - 1,-1):
            for j in range(i):
                aux = word[j:i]
                if englishDictionary.check(aux) and not hasNumeric(aux) and len(aux) > 1:
                    return removeDots(aux), True
                else:
                    failure = True
    
    if failure:  
        return word, False
        

In [150]:
def mapBinaryLabel(binaryIndicator, labelsList):
    
    """ 
    RECIBE: 1 o 0
    DEVUELVE: labelsList[0] o labelsList[1] respectivamente
    """  
    
    if binaryIndicator == 1:
        return labelsList[0]
    if binaryIndicator == 0:
        return labelsList[1]
    else:
        return None

In [212]:
def generateAbsolutePath(fileName, exists = False, nombreRepo = 'DatosRepo'):
    
    """ 
    RECIBE: El nombre de un archivo
            exists es para indicar que el archivo ya existe. En cuyo caso lo va a buscar a donde este
    DEVUELVE: El path absoluto del archivo
    TENER EN CUENTA: Si se pone mal escrito el archivo es posible que lo encuentre igual por ej 'trai' en vez de 'train.csv'
                     Asume que tu repo se llama DatosRepo a menos que se lo digas en <nombreRepo>
    EJEMPLO: 'miArchivo.txt' -----> '/home/axelmpm/DatosRepo/miArchivo.txt'
    """
    
    path = '/home/' + getpass.getuser() + '/' + nombreRepo
    
    if exists:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in [f for f in filenames if (os.path.basename(f) == fileName)]:
                return os.path.join(dirpath, filename)
    else:
        return os.path.abspath("") + '/' + fileName

In [152]:
def readDic(fileName, transformStringedListToList = False):
    
    """ 
    RECIBE: El nombre de un archivo .csv
    DEVUELVE: El diccionario que estaba almacenado en disco
    """
    
    df = pd.read_csv(generateAbsolutePath(fileName + '.csv'))
    
    if transformStringedListToList:
        df['values'] = df.apply(lambda row : ast.literal_eval(row['values']), axis = 1)
    
    dic = {}
    
    keys = list(df['keys'])
    values = list(df['values'])
    
    for i in range(len(keys)):
        dic[keys[i]] = values[i]
    
    return dic

In [153]:
def getWordInListFromFile(fileName):
    
    """ 
    RECIBE: El nombre de un archivo donde cada linea tiene palabras
    DEVUELVE: Una lista donde cada elemento son las palabras que estaban como linea en el archivo (no toma las palabras de longitud 1)
    """ 
    
    file = open(generateAbsolutePath(fileName),'r')
    
    l = []
    
    for line in file:
        if len(line[:-1]) > 1:
            l.append(line[:-1]) #le saco el \n
    file.close()
    return l

In [154]:
def hasCertainWords(stuffWithWordsInIt,words):
    
    """ 
    RECIBE: Recibe algo que tiene palabras dentro (texto, lista de palabras, conjuntos con palabras, etc)
    DEVUELVE: Devuelve True o False dependiendo de si el el contendor contiene ALGUNA palabra en la lista de palabras
    """ 
    
    for word in words:
        if word in stuffWithWordsInIt:
            return True
    return False

In [155]:
def clasifyWordsRespectToEnglish(bagOfWords):
    
    """ 
    RECIBE: Un contenedor de palabras que podrian o no estar en ingles
    DEVUELVE: Una lista de palabras que estan en ingles y otra que no estaban en ingles
    EJEMPLO: {'hello..gggggg,ghghghgh,who,hola'} -------> {hello,who}, {ghghghgh,hola}
    """ 
    
    bagOfEnglishWords = []
    bagOfNonEnglishWords = []
    for word in bagOfWords:
        processedWord,isEnglish = getEnlgishWord(word)
        if isEnglish:
            bagOfEnglishWords.append(processedWord)
        else:
            bagOfNonEnglishWords.append(processedWord)
        
    return bagOfEnglishWords, bagOfNonEnglishWords

In [156]:
def getDataFrameOfFrecuencies(elementsLabel,elements, sort = True, sortAscending = False):
    
    """ 
    RECIBE: Una lista o secuencia de elementos y un nombre para estos
    DEVUELVE: Un dataframe con dos columnas, una de labels y otro de las frecuencias
    """ 
    
    temp_elements = list(elements.copy())
    frecs_dict = {i:temp_elements.count(i) for i in temp_elements}

    df = pd.DataFrame({elementsLabel : list(frecs_dict.keys()), 'Cantidad' : list(frecs_dict.values())})
    
    if sort:
        df.sort_values('Cantidad', ascending = sortAscending, inplace = True)
    return df

In [157]:
def dicToDataFrame(dic, keysLabel = "keys", valuesLabel = "values", sort = True, sortAscending = False):
    
    """ 
    RECIBE: Un diccionario y nombres para sus claves y valores y si si quiere ordenar o no ascendente o no
    DEVUELVE: Un dataframe con dos columnas, una de keys y otro de las values
    """
    
    df = pd.DataFrame({keysLabel : list(dic.keys()), valuesLabel : list(dic.values())})
    
    if sort:
        df.sort_values(valuesLabel, ascending = sortAscending, inplace = True)
    return df

In [158]:
def getHammingSimilarity(word1,word2):
    
    """ 
    RECIBE: Dos palabras de igual longitud
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    EJEMPLO: word1 = casa, word2 = capa ---->  0.75 (porque tienen en comun la primera c, la segunda a
             y la ultima a, osea 3/4 bien)
    """     
    
    length = len(word1)
    mismatches = 0
    
    for i in range(length):
        if word1[i] != word2[i]:
            mismatches += 1
    return 1 - mismatches/length
    

In [159]:
def getAllConstantSum(k,n):

    """ 
    RECIBE: k es el valor a sumar de forma constante, n es la cantidad de numeros a sumar
    DEVUELVE: Todas las listas de n numeros que sumados dan k 
    """ 
    
    if n == 1:
        return [[k]]
    
    sums = []
    
    for i in range(k + 1):
        
        subsums = getAllConstantSum(k - i, n -1)
        
        for subsum in subsums:
            sums.append([i] + subsum)
    
    return sums

In [160]:
def generateWordExtentionsIntoNChars(word,n,extentionChar):
    
    """ 
    RECIBE: Una palabra a extender, la cantidad de caracteres que debe tener la version extendida y que caracter usar al extender
    DEVUELVE: Devuelve un set de todas las posibles extensiones
    EJEMPLO: word = 'casa', n = 6, extentionChar = '.' ---->
                {'..casa','.c.asa','.ca.sa','.cas.a','.casa.','c..asa','c.a.sa','c.as.a','c.asa.','ca..sa'
                    'ca.s.a','ca.sa.','cas..a','cas.a.','casa..'}   
    """
    
    if len(word) == '':
        return [extentionChar * n]
    
    if (len(word) > n):
        return [word]
    
    if len(word) == n:
        return [word]
    
    wordLength = len(word)
    amountOfCharsToAdd = n - wordLength
    
    extentions = set()
    
    amounts = getAllConstantSum(amountOfCharsToAdd,wordLength + 1)
    
    for amountOfExtendingChars in amounts:

        extention = ''
        for i in range(len(word)):
            extention += (extentionChar * amountOfExtendingChars[i]) + word[i]
        extention += extentionChar * amountOfExtendingChars[-1]
        
        extentions.add(''.join(list(extention)))
    
    return extentions

In [161]:
def getSimilarity(text1,text2):
    
    """ 
    RECIBE: Dos textos de cualquier longitud (no muy largos)
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    A TENER EN CUENTA: Usa la similaridad de Hamming
                       Es mas o menos costosa a partir de 20 caracteres de texto aprox
                       Remueve los espacios por lo que no hace falta preprocesar texto
    EJEMPLO: word1 = casa, word2 = ca ---->  0.5 (porque tienen en comun la primera c, la segunda a, osea 2/4 bien)
    """ 
    
    EMPTY_CHARACTER = ' '
    
    if text1 + text2 == '':
        return 1
    
    if text1 + text2 == text1 or text1 + text2 == text2:
        return 0
    
    splitedText1 = text1.split(' ')
    splitedText2 = text2.split(' ')
    
    concatenatedWord1 = (''.join(splitedText1)).lower()
    concatenatedWord2 = (''.join(splitedText2)).lower()
    
    if len(concatenatedWord1) > len(concatenatedWord2):
        longest = concatenatedWord1
        shortest = concatenatedWord2
    else:
        longest = concatenatedWord2
        shortest = concatenatedWord1
        
    lenDif = len(longest) - len(shortest)
    maxSimilarity = 0
    extentions = generateWordExtentionsIntoNChars(shortest,len(longest),EMPTY_CHARACTER)
    
    for extention in extentions:
        
        subSimilarity = getHammingSimilarity(extention,longest)
        #print(shortest,subWord,longest,subSimilarity)
        
        if (subSimilarity >= maxSimilarity):
            maxSimilarity = subSimilarity
            
    return maxSimilarity
        

In [162]:
def getMaxSimilarityWord(word,words):

    """ 
    RECIBE: El una palabra y palabras a comparar
    DEVUELVE: La palabra que mas se parece
    """
    
    maxSimilarity = 0
    mostSimilarKnownWord = ''
    
    if word in words:
        return word
    
    candidateWords = []
    for w in words:
        if len(w) > 0 and w[0].lower() == word[0].lower():
            candidateWords.append(w)
    
    for knownWord in candidateWords:
        
        similarity = getSimilarity(word,knownWord)

        if maxSimilarity <= similarity:

            maxSimilarity = similarity
            mostSimilarKnownWord = knownWord
            
    return mostSimilarKnownWord

In [163]:
def isSimilar(text1, text2, threshold = 0.7):

    """ 
    RECIBE: Dos textos (no muy grandes)
    DEVUELVE: Si la similitud es mayor a un threshold
    """ 
    
    return getSimilarity(text1,text2) >= threshold

In [164]:
def isIn(element,container):

    """ 
    RECIBE: Un elemento y un contenedor de elementos
    DEVUELVE: Si el elemento esta o no en el contenedor
    """ 
    
    return element in container

In [165]:
def acortar_texto(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna con version corta de la columna 'text'
    """
    
    
    df['short_text'] = 'None'
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['short_text'][y:]= df.iloc[y][0][:15] + "..."

In [166]:
def isQuestion(text):
    '''
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles seguida por al menos un signo de pregunta, sino retorna FALSE
    '''
    text = text.replace('?', ' ? ')
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text, specialCharacters = [',','.','#','!','%','*','(',')','"','-','_', '$', ':', '/', '\''])
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            for j in range(textSplitted.index(word) + 1, len(textSplitted)):
                if ('?' in textSplitted[j]):
                    return True
    return False

In [167]:
def takeOutSpecialCaractersFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: El string sin caracteres especiales. Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')','"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    
    
    for char in specialCharacters:
        text = text.replace(char, ' ')
    return text

In [168]:
def separateOutBiaSpecialCaractersAndMayuscFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: Una lista de los substrings que fuerons separados por los caracteres especiales o las mayusuclas.
               Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')',
                             '"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    
    words = []
    subWord = ''
    prevC = ''
    
    for c in text:
        
        if c.isupper() and prevC.islower():
            words.append(subWord)
            subWord = c
        else:        
            if c in specialCharacters:
                if subWord != '':
                    words.append(subWord)
                    subWord = ''
            else:
                subWord += c
        prevC = c
        
    if subWord != '':
        words.append(subWord)
        
    return words

In [169]:
def barplot(x,y,data,xLabel = None,yLabel = None,title = '',
            palette = 'Spectral',figX = 25,figY = 10,fontSize = 1.7):
    
    '''
    RECIBE:
        x: es el label de la columna con los datos de x del dataFrame 'data'
        y: es el label de la columna con los datos de y del dataFrame 'data'
        data: es el dataframe de donde sacar los datos
        xLabel: es el label para x en el plot
        yLabel: es el label para y en el plot
        xLabel: es el titulo del plot
        palette: es la paleta de colores a usar
        figX: es el tamaño en x del plot
        figY: es el tamaño en y del plot
        fontSize: es el tamaño del font en el plot
    A TENER EN CUENTA: No afecta al dataframe 'data'
    DEVUELVE: El plot
    '''
    
    if xLabel == None:
        xLabel = x
    
    if yLabel == None:
        yLabel = y
    
    sns.set(rc = {'figure.figsize':(figX,figY)})
    sns.set(font_scale = fontSize)
    v = sns.barplot(x = x, y = y, data = data, 
                 palette = palette, saturation = 1)
    
    v.set(xlabel = xLabel, ylabel= yLabel)
    v.set(title = title)
    
    return v

In [170]:
def addContextualKnowledge(wordList,context):

    """ 
    RECIBE: Una lista de palabras y un contexto de textos previos
    DEVUELVE: El contexto actualizado con el contenido del texto pasado
    """
    
    words = list(set(wordList))
    pairs = []
    
    if len(words) == 1:
        pairs.append((words[0],words[0]))
    
    else:
        for i in range(len(words)):
            for j in range(i + 1,len(words)):
                if j < len(words):
                    pairs.append((words[i],words[j]))
    
    for pair in pairs:
        
        if pair in context:
            context[pair] += 1
        else:
            context[pair] = 1
            
    return context

In [171]:
def getContextualKnowledge(texts):

    """ 
    RECIBE: Una lista de listas de palabras (lista de tweets)
    DEVUELVE: Un diccionario llamado contexto que tiene como claves combinaciones de palabras que se dieron
                juntas en algun texto de la lista y cuya clave es la cantidad de textos en los que se dio
    """
    
    context = {}
    
    for wordList in texts:
        
        context = addContextualKnowledge(wordList,context)
    
    return context

In [172]:
def getPairsWith(word,context):

    """ 
    RECIBE: Una palabra y un contexto
    DEVUELVE: Devuelve los pares de palabras del contexto que contienen a la palabra pasada
    """
    
    pairsWith = []
    
    for pair in context:
        
        if word in pair:
            pairsWith.append(pair)
    
    return pairsWith

In [173]:
def getAsociatedWordsWithReinforcement(word,context):

    """ 
    RECIBE: Una palabra y un contexto
    DEVUELVE: Una lista de (palabra,reinforcements) que seria como las palabras y la frecuencia con la que
                las vio juntos a word y a palabra
    """
    
    asociatedPairs = getPairsWith(word,context)
    asociatedWordAndReinforcement = []

    for asociatedPair in asociatedPairs:
        
        if word != asociatedPair[0]:
            asociatedWord = asociatedPair[0]
        else:
            asociatedWord = asociatedPair[1]

        asociatedWordAndReinforcement.append((asociatedWord,context[asociatedPair]))
  
    return asociatedWordAndReinforcement

In [174]:
def getContextualMap(context):
    
    """ 
    RECIBE: Un contexto
    DEVUELVE: Un diccionario que contiene como clave todas las palabras del contexto y como value sus asociadas
                con su reinforcement
    """
    
    contextualMap = {}
    
    for pair in context:
        
        for word in pair:
            
            if not word in contextualMap:

                asociatedWordAndReinforcement = getAsociatedWordsWithReinforcement(word,context)

                contextualMap[word] = asociatedWordAndReinforcement
            
    return contextualMap

In [175]:
def mergeIntoWords(texts):
    
    """ 
    RECIBE: Una lista de lista de palabras
    DEVUELVE: Una lista de las palabras de los textos recibidos
    """
    return list(set(itertools.chain.from_iterable(texts)))                
        

In [176]:
def cleanUpWords(texts):
    
    """ 
    RECIBE: Una lista de textos
    DEVUELVE: Una lista de lista de palabras (lista de tweets) cuyas palabras de los textos recibidos no son vacias
    """
    
    cleanedTexts = []
    
    stopWords =  set(stopwords.words('english'))
    
    for text in texts:
    
        words = text.split(' ')
        cleanedWords = []
        
        for word in words:
            
            subWords = separateOutBiaSpecialCaractersAndMayuscFromText(word) 
            
            for subWord in subWords:
                
                if subWord == '':
                    continue
                else:
                    cleanedWords.append(subWord.lower())
        
        cleanedTexts.append(cleanedWords)
        
    return cleanedTexts

In [177]:
def furtherCleanUp(words,stopWords):

    """ 
    RECIBE: Una lista de palabras y unas stopwords
    DEVUELVE: Una lista de palabras que no tienen longitud menor a 3 ni son stopwords
    """
    
    cleanWords = []
    
    for word in words:
        
        if not (len(word) < 3 or word in stopWords):
            cleanWords.append(word)
            
    return cleanWords

In [283]:
def truncateMapTo(contextualMap,words):
    
    """ 
    RECIBE: Toma un contextual map y unas palabras
    DEVUELVE: Un contextual map cuyas palabras en key y en los values no estan en la lista de palabras recibidas
    """
    
    truncatedContextualMap = {}
    
    for word in contextualMap:
        
        if word in words:
            asociates = contextualMap[word]

            filteredAsociates = []

            i = 0
            checksForEveryWordInAsociates = [pair[0] in words for pair in asociates]

            for asociatedWordIsIn in checksForEveryWordInAsociates:

                if asociatedWordIsIn:
                    filteredAsociates.append(asociates[i])
                i += 1

            truncatedContextualMap[word] = filteredAsociates
    return truncatedContextualMap

In [307]:
def readFromFileIntoList(fileName, strings = False):
    
    """ 
    RECIBE: Un nombre de archivo que se supone que almacena una lista previamente almacenada 
            con el metodo listToFile
    DEVUELVE: La lista guardada en ese archivo
    """
    
    file = open(generateAbsolutePath(fileName),'r')
    
    strList = (file.read()).split('&&&')[:-1]
    
    recoveredList = []
    
    for e in strList:
        
        if strings:
            recoveredList.append(e)
        else:
            recoveredList.append(ast.literal_eval(e))
        
    return recoveredList
    

In [286]:
def listToFile(aList,fileName):

    """ 
    RECIBE: Un nombre para el archivo que va a almacenar la lista y 
            una lista a almacenar
    """
    
    file = open(generateAbsolutePath(fileName),'w')
    
    for e in aList:
        file.write(str(e) + '&&&')
        
    file.close()    

In [313]:
class Context:
    
    """ 
    Clase que engloba toda las operaciones y nociones de contexto de un texto
    """
    
    def __init__(self,data = None, similarityThreshold = 0.8, load = None):
        
        """ 
        RECIBE: 
            texts es una lista de textos (tweets) a entender
            similarityThreshold es una tolerancia para la similaridad. 
                                Si dos palabras superan ese limite se consideran simialres
            load es opcional y es el nombre del archivo que almacena
                                el contextualMap guardado en disco en caso de haberlo hecho
        """
        
        self.stopWords = set(stopwords.words('english'))
        self.similarityThreshold = similarityThreshold
        
        if load == None:
            
            self.texts = cleanUpWords(texts)        
            self.words = mergeIntoWords(self.texts)
            self.wordNet = getContextualKnowledge(self.texts)
            self.englishWords, self.nonEnglishWords = clasifyWordsRespectToEnglish(self.words)
            self.nouns = furtherCleanUp(self.englishWords,self.stopWords)
            self.contextualMap = getContextualMap(self.wordNet)
            self.englishContextualMap = truncateMapTo(self.contextualMap,self.nouns)
            
        else:

            __atrbt_texts = load + '_' + 'texts'
            __atrbt_words = load + '_' + 'words'
            __atrbt_wordNet = load + '_' + 'wordNet'
            __atrbt_englishWords = load + '_' + 'englishWords'
            __atrbt_nonEnglishWords = load + '_' + 'nonEnglishWords'
            __atrbt_nouns = load + '_' + 'nouns'
            __atrbt_contextualMap = load + '_' + 'contextualMap'
            __atrbt_englishContextualMap = load + '_' + 'englishContextualMap'

            self.texts = readFromFileIntoList(__atrbt_texts)        
            self.words = readFromFileIntoList(__atrbt_words, strings = True)
            self.wordNet = readDic(__atrbt_wordNet)
            self.englishWords = readFromFileIntoList(__atrbt_englishWords, strings = True)
            self.nonEnglishWords = readFromFileIntoList(__atrbt_nonEnglishWords, strings = True)
            self.nouns = readFromFileIntoList(__atrbt_nouns, strings = True)
            self.contextualMap = readDic(__atrbt_contextualMap, transformStringedListToList = True)
            self.englishContextualMap = readDic(__atrbt_englishContextualMap, transformStringedListToList = True)           
        
        
    def related(self, word, r = 1, restrict = True):
        
        """ 
        RECIBE: La palabra de la que se quiere conocer sus asociados y el orden minimo de su asociacion (su reinforcement)
                restric es para quedarse solo con las relaciones con palabras filtradas
        DEVUELVE: Una lista de todos los asociados (palabras, orden) que cumplen con el r
        """
        
        if restrict:
            asociated = self.englishContextualMap[getMaxSimilarityWord(word,self.words)]
        else:
            asociated = self.contextualMap[getMaxSimilarityWord(word,self.words)]
        
        if r == 1:
            return asociated
        
        strongAsociated = []
        
        for pair in asociated:
            if pair[1] >= r:
                strongAsociated.append(pair)
                                
        return strongAsociated
    
    
    def save(self,name):
        
        """ 
        RECIBE: El nombre de archivo que guarda el contextualMap
        DEVUELVE: Guarda el contextualMap en un csv
        """
        
        listToFile(self.texts, name + '_' + 'texts')
        listToFile(self.words, name + '_' + 'words')       
        listToFile(self.englishWords, name + '_' + 'englishWords')
        listToFile(self.nonEnglishWords, name + '_' + 'nonEnglishWords')
        listToFile(self.nouns, name + '_' + 'nouns')
        
        dicToDataFrame(self.wordNet).to_csv(name + '_' + 'wordNet' + '.csv')
        dicToDataFrame(self.contextualMap).to_csv(name + '_' + 'contextualMap' + '.csv')
        dicToDataFrame(self.englishContextualMap).to_csv(name + '_' + 'englishContextualMap' + '.csv')
        
        

In [133]:
def getNegativity(text,negativeWords):
    
    """ 
    RECIBE: Un texto y un conjunto de palabras negativas
    DEVUELVE: Un numero de 0 a 1 donde 1 es completamente negativo y 0 es nada negativo
    """
    
    words = text.split(' ')
    
    count = 0
    for word in words:
        for negativeWord in negativeWords:
        
            if isSimilar(word,negativeWord):
                count += 1
                break
                
    return count/len(words)

In [134]:
#Estaria bueno ponderar las palabras negativas de alguna forma con un diccionario

In [135]:
#Estoy implicando que mas negativo es un tweet cuando mas palabras negativas usa pero no por su peso

#'robo robo robo robo robo' es mas negativo que 'en un terremoto en japon muerieron treinticinco millones de personas'

In [136]:
#hay palabras que aumentan su peso por el contexto (otras palabras que estan) y hay palabras que ganan negatividad cuando hay otras

#por ej personas no es negativo, pero si esta en la misma frase que muerieron, si lo es

In [137]:
#asociar palabras con palabras a las que "contagia" de negatividad de un diccionario

In [138]:
negs = ['romper','cortar','cuidado','robo','muerieron','terremoto']

In [139]:
text = 'robo robo robo robo robo'1
getNegativity(text,negs)

1.0

In [140]:
text = 'en un terremoto en japon muieron treinticinco millones de personas'
getNegativity(text,negs)

0.2